# Data split
En esta sección vamos a hacer el split en train y test para nuestros futuros experimentos. El resultado van a ser 6 archivos. Dos archivos para train, dos para validacion y dos para test con X e y.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('../data-new/data-selected-features.csv')

In [3]:
df.head()

,I RENAL AGUDA,NUMERO VASOS,SANGRADO MAYOR*,FUNCION VENTRICULAR IZQ,CRM,CREAT,KILLIP Ingreso,GB,Peor KILLIP,INOTROPICOS,TAS INGRESO,GLUCEMIA INGR,MUERTE HOSP
0,0.0,2.0,0.0,1.0,1.0,0.58,1,8920.0,1,0.0,120.0,160.0,0
1,0.0,0.0,0.0,2.0,0.0,1.06,1,10340.0,1,0.0,140.0,131.0,0
2,0.0,1.0,0.0,1.0,0.0,0.83,1,6780.0,1,0.0,150.0,98.0,0
3,0.0,1.0,0.0,1.0,0.0,1.06,1,5830.0,1,0.0,120.0,106.0,0
4,0.0,1.0,0.0,1.0,0.0,0.85,1,14990.0,1,0.0,120.0,110.0,0


In [4]:
target = 'MUERTE HOSP'

Vamos a hacer el split en train, validation y test. El porcentaje sera del 70% para train, 15% para validation y 15% para test. Esto nos deja con suficientes datos para entrenar y validar nuestros modelos.

El resultado de este split va a ser 6 archivos. Dos archivos para train y test con X e y. Dos archivos para validation y test con X e y.

In [5]:
X = df.drop(columns=target)
y = df[[target]]

# First split: separate test set (15%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, 
    test_size=0.15,
    random_state=42,
    stratify=y
)

# Second split: from remaining 85%, get 70/15 ratio
# To get 70% total from the 85% remaining, we need test_size = 15/85 ≈ 0.176
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp,
    test_size=(15/85),  # This ensures we get 15% of total from the remaining 85%
    random_state=42,
    stratify=y_temp
)

In [6]:
# Verify the proportions
total_samples = len(X)
print(f"Total samples: {total_samples}")
print(f"Training samples: {len(X_train)} ({len(X_train)/total_samples:.1%})")
print(f"Validation samples: {len(X_val)} ({len(X_val)/total_samples:.1%})")
print(f"Test samples: {len(X_test)} ({len(X_test)/total_samples:.1%})")

Total samples: 1818
Training samples: 1272 (70.0%)
Validation samples: 273 (15.0%)
Test samples: 273 (15.0%)


In [7]:
# Ver duplicados dentro de cada conjunto
print(f"Duplicados en X_train: {X_train.duplicated().sum()}")
print(f"Duplicados en X_val: {X_val.duplicated().sum()}")
print(f"Duplicados en X_test: {X_test.duplicated().sum()}")


Duplicados en X_train: 0
Duplicados en X_val: 0
Duplicados en X_test: 0


In [8]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(1272, 12)
(273, 12)
(273, 12)
(1272, 1)
(273, 1)
(273, 1)


In [9]:
# Remove duplicates between X_train and X_val, and drop from y_val accordingly
mask_val = ~X_val.isin(X_train.to_dict(orient='list')).all(axis=1)
X_val = X_val[mask_val]
y_val = y_val[mask_val]

# Remove duplicates between X_train and X_test, and drop from y_test accordingly
mask_test_train = ~X_test.isin(X_train.to_dict(orient='list')).all(axis=1)
X_test = X_test[mask_test_train]
y_test = y_test[mask_test_train]

# Remove duplicates between X_val and X_test, and drop from y_test accordingly
mask_test_val = ~X_test.isin(X_val.to_dict(orient='list')).all(axis=1)
X_test = X_test[mask_test_val]
y_test = y_test[mask_test_val]

# Ensure no internal duplicates in each set
X_train, y_train = X_train.drop_duplicates(), y_train.loc[X_train.index]
X_val, y_val = X_val.drop_duplicates(), y_val.loc[X_val.index]
X_test, y_test = X_test.drop_duplicates(), y_test.loc[X_test.index]

# Print final dataset shapes
print(f"New X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"New X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"New X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


New X_train shape: (1272, 12), y_train shape: (1272, 1)
New X_val shape: (112, 12), y_val shape: (112, 1)
New X_test shape: (98, 12), y_test shape: (98, 1)


In [11]:
# Save the splits
X_train.to_csv("../data-new/train/X_train.csv", index=False)
X_val.to_csv("../data-new/val/X_val.csv", index=False)
X_test.to_csv("../data-new/test/X_test.csv", index=False)

y_train.to_csv("../data-new/train/y_train.csv", index=False)
y_val.to_csv("../data-new/val/y_val.csv", index=False)
y_test.to_csv("../data-new/test/y_test.csv", index=False)